### Restart and Run All

In [1]:
import calendar
import pandas as pd
import numpy as np
import panel as pn
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import show
pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

today = date.today()
today

datetime.date(2022, 9, 24)

### Tables in the process

In [2]:
cols = 'trade name qty price active reason market xdate'.split()
colt = 'trans name spd reason qty target current percent active xdate'.split()

format_dict = {
    'qty':'{:,}','available_qty':'{:,}',
    'price':'{:.2f}','cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}',
    'buy_target':'{:.2f}','sell_target':'{:.2f}','target':'{:.2f}','current':'{:.2f}',
    'beta':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}','volume':'{:,.2f}',
    'percent':'{:,.2f}',
    'shares':'{:,}',    
    'q4':'{:.4f}','q3':'{:.4f}','q2':'{:.4f}','q1':'{:.4f}','dividend':'{:.4f}',
    'xdate':'{:%Y-%m-%d}','paiddate':'{:%Y-%m-%d}',
}

In [3]:
sql = '''
SELECT *
FROM orders
ORDER BY id DESC
LIMIT 1'''
orders = pd.read_sql(sql, conlite)
orders

,id,trade,name,qty,price,active,reason,market,xdate
0,20,S,WHART,10000,11.1,1,C10.5,SET,2022-08-18


In [4]:
names = orders['name']
name = names.to_string(index=False)

In [5]:
sql = '''
SELECT *
FROM dividend
WHERE name = '%s'
'''
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,WHART,0.1195,0.2553,0.1920,0.1915,0.7583,"30,000",2022-08-18,2022-09-02,,0


In [6]:
sql = """
SELECT * FROM stocks WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * FROM stocks WHERE name = 'WHART'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,687,WHART,14.08,10.80,I,0.00,11.10,24.74,0.21,11.70,"10,000",-5,4,"30,000",0,0,C10.5,SET


### Create orders from stocks after adjust stocks in port_lite from consensus

In [7]:
sql = '''
DELETE FROM orders'''
rp = conlite.execute(sql)
rp.rowcount

20

In [8]:
sql = '''
SELECT status AS trade, name, qty, buy_target AS price, 0 AS active, reason, market
FROM stocks
WHERE status IN ('B','O')
ORDER BY name'''
df_buy = pd.read_sql(sql, conlite)
df_buy

,trade,name,qty,price,active,reason,market
0,O,ASK,3000,33.5,0,52WL,SET
1,O,BAM,6000,16.0,0,XXX,SET100
2,O,BANPU,7500,13.0,0,CP1S,SET50
3,O,BCP,3000,29.0,0,XXX,SET100
4,O,BLA,4000,33.0,0,1L,SET50
5,B,CPNREIT,10000,18.8,0,HD,SET
6,O,JMART,3000,48.0,0,1L,SET50
7,O,JMT,1200,73.0,0,1L,SET50
8,O,KSL,30000,3.3,0,6L,SET
9,O,LANNA,3000,20.8,0,3L,SET


In [9]:
sql = '''
SELECT status AS trade, name, qty, sell_target AS price, 0 AS active, reason, market
FROM stocks
WHERE status IN ('I','S')'''
df_sell = pd.read_sql(sql, conlite)
df_sell

,trade,name,qty,price,active,reason,market
0,S,MCS,25000,12.90,0,C12.8,SET
1,S,PTTGC,3000,51.50,0,C51.25,SET50
2,I,JASIF,10000,9.20,0,CP1S,SET
3,S,STA,2500,26.00,0,CP1S,SET100
4,I,BCH,3000,20.60,0,C20.50,SET100
5,I,DOHOME,2800,20.30,0,RD20pct,SET100
6,I,WHART,10000,11.10,0,C10.5,SET
7,S,ASP,10000,3.82,0,CP1S,SET
8,I,SCCC,1200,159.00,0,CUT,SET
9,I,IVL,2400,44.25,0,DOS,SET50


In [10]:
df_order = df_buy.append(df_sell, ignore_index=True)
df_order.shape

(30, 7)

In [11]:
df_order.groupby(['trade']).count()

,name,qty,price,active,reason,market
trade,,,,,,
B,1,1,1,1,1,1
I,8,8,8,8,8,8
O,17,17,17,17,17,17
S,4,4,4,4,4,4


In [12]:
df_order.loc[df_order.trade == 'I','trade'] = 'S'
df_order.groupby(['trade']).count()

,name,qty,price,active,reason,market
trade,,,,,,
B,1,1,1,1,1,1
O,17,17,17,17,17,17
S,12,12,12,12,12,12


In [13]:
sql = '''
SELECT name, xdate
FROM dividend
'''
dividend = pd.read_sql(sql, const)
dividend.columns = dividend.columns.str.lower()
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend.shape

(86, 2)

In [14]:
df_merge = pd.merge(df_order, dividend, on='name', how='outer', indicator=True)
df_merge.shape

(94, 9)

### If there is no dividend record, must create dividend record before proceed

In [15]:
no_div = df_merge[df_merge['_merge'] == 'left_only']
no_div

,trade,name,qty,price,active,reason,market,xdate,_merge
3,O,BCP,3000.0,29.0,0.0,XXX,SET100,NaT,left_only
8,O,KSL,30000.0,3.3,0.0,6L,SET,NaT,left_only
9,O,LANNA,3000.0,20.8,0.0,3L,SET,NaT,left_only
10,O,OISHI,2400.0,42.0,0.0,6L,SET,NaT,left_only
11,O,PSL,6000.0,14.0,0.0,3L,SET100,NaT,left_only
13,O,SINGER,3000.0,35.5,0.0,52WL,SET100,NaT,left_only
15,O,SSP,6000.0,8.5,0.0,52WL,SET,NaT,left_only
17,O,TOP,3000.0,54.0,0.0,1L,SET50,NaT,left_only


### Rerun from here

In [16]:
# Include both having diviedend and having no dividend records
mask = ((df_merge['_merge'] == 'both') | (df_merge['_merge'] == 'left_only'))
df_orders = df_merge[mask].copy()
df_orders.drop(['_merge'], axis=1, inplace=True)
df_orders['qty'] = df_orders['qty'].astype('int64')
df_orders['active'] = df_orders['active'].astype('int64')
df_orders['xdate'] = df_orders['xdate'].astype('str')
df_orders.groupby('trade').count()

,name,qty,price,active,reason,market,xdate
trade,,,,,,,
B,1,1,1,1,1,1,1
O,17,17,17,17,17,17,17
S,12,12,12,12,12,12,12


### Set xdate 2 weeks window of first Tuesday and second Friday

In [17]:
fm_date = '2022-05-03'

### Restart and Run All Cells

In [18]:
fltr_sts_o = (df_orders.trade == 'O') 
df_sts_o = df_orders[fltr_sts_o]
df_sts_o

,trade,name,qty,price,active,reason,market,xdate
0,O,ASK,3000,33.5,0,52WL,SET,2022-03-03
1,O,BAM,6000,16.0,0,XXX,SET100,2022-04-29
2,O,BANPU,7500,13.0,0,CP1S,SET50,2022-09-14
3,O,BCP,3000,29.0,0,XXX,SET100,NaT
4,O,BLA,4000,33.0,0,1L,SET50,2022-08-23
6,O,JMART,3000,48.0,0,1L,SET50,2022-04-19
7,O,JMT,1200,73.0,0,1L,SET50,2022-04-19
8,O,KSL,30000,3.3,0,6L,SET,NaT
9,O,LANNA,3000,20.8,0,3L,SET,NaT
10,O,OISHI,2400,42.0,0,6L,SET,NaT


In [19]:
fltr_sts_b = (df_orders.trade == 'B') 
df_sts_b = df_orders[fltr_sts_b]
df_sts_b

,trade,name,qty,price,active,reason,market,xdate
5,B,CPNREIT,10000,18.8,0,HD,SET,2022-08-17


In [20]:
fltr_sts_s = (df_orders.trade == 'S')
df_sts_s = df_orders[fltr_sts_s]
df_sts_s.sort_values('name',ascending=True)

,trade,name,qty,price,active,reason,market,xdate
25,S,ASP,10000,3.82,0,CP1S,SET,2022-08-24
22,S,BCH,3000,20.60,0,C20.50,SET100,2022-08-31
23,S,DOHOME,2800,20.30,0,RD20pct,SET100,2022-02-02
28,S,GVREIT,10000,9.20,0,DOS,SET,2022-08-26
27,S,IVL,2400,44.25,0,DOS,SET50,2022-08-30
20,S,JASIF,10000,9.20,0,CP1S,SET,2022-08-19
18,S,MCS,25000,12.90,0,C12.8,SET,2022-09-03
19,S,PTTGC,3000,51.50,0,C51.25,SET50,2022-09-02
26,S,SCCC,1200,159.00,0,CUT,SET,2022-08-13
21,S,STA,2500,26.00,0,CP1S,SET100,2022-08-23


In [21]:
frames = [df_sts_b, df_sts_o, df_sts_s]
df_orders = pd.concat(frames, sort=False)
df_orders.sort_values(['trade','name'],ascending=True)

,trade,name,qty,price,active,reason,market,xdate
5,B,CPNREIT,10000,18.80,0,HD,SET,2022-08-17
0,O,ASK,3000,33.50,0,52WL,SET,2022-03-03
1,O,BAM,6000,16.00,0,XXX,SET100,2022-04-29
2,O,BANPU,7500,13.00,0,CP1S,SET50,2022-09-14
3,O,BCP,3000,29.00,0,XXX,SET100,NaT
4,O,BLA,4000,33.00,0,1L,SET50,2022-08-23
6,O,JMART,3000,48.00,0,1L,SET50,2022-04-19
7,O,JMT,1200,73.00,0,1L,SET50,2022-04-19
8,O,KSL,30000,3.30,0,6L,SET,NaT
9,O,LANNA,3000,20.80,0,3L,SET,NaT


In [22]:
df_orders.loc[df_orders.trade == 'O','trade'] = 'B'
df_orders.groupby(['trade']).count()

,name,qty,price,active,reason,market,xdate
trade,,,,,,,
B,18,18,18,18,18,18,18
S,12,12,12,12,12,12,12


In [23]:
rcds = df_orders.values.tolist()
len(rcds)

30

In [24]:
for rcd in rcds:
    print(rcd)

['B', 'CPNREIT', 10000, 18.8, 0, 'HD', 'SET', '2022-08-17']
['B', 'ASK', 3000, 33.5, 0, '52WL', 'SET', '2022-03-03']
['B', 'BAM', 6000, 16.0, 0, 'XXX', 'SET100', '2022-04-29']
['B', 'BANPU', 7500, 13.0, 0, 'CP1S', 'SET50', '2022-09-14']
['B', 'BCP', 3000, 29.0, 0, 'XXX', 'SET100', 'NaT']
['B', 'BLA', 4000, 33.0, 0, '1L', 'SET50', '2022-08-23']
['B', 'JMART', 3000, 48.0, 0, '1L', 'SET50', '2022-04-19']
['B', 'JMT', 1200, 73.0, 0, '1L', 'SET50', '2022-04-19']
['B', 'KSL', 30000, 3.3, 0, '6L', 'SET', 'NaT']
['B', 'LANNA', 3000, 20.8, 0, '3L', 'SET', 'NaT']
['B', 'OISHI', 2400, 42.0, 0, '6L', 'SET', 'NaT']
['B', 'PSL', 6000, 14.0, 0, '3L', 'SET100', 'NaT']
['B', 'PTT', 4500, 35.0, 0, '1L', 'SET50', '2022-09-28']
['B', 'SINGER', 3000, 35.5, 0, '52WL', 'SET100', 'NaT']
['B', 'SPALI', 6000, 18.5, 0, '3L', 'SET100', '2022-08-23']
['B', 'SSP', 6000, 8.5, 0, '52WL', 'SET', 'NaT']
['B', 'STARK', 25000, 3.9, 0, '3L', 'SET100', '2022-02-02']
['B', 'TOP', 3000, 54.0, 0, '1L', 'SET50', 'NaT']
['S', '

In [25]:
sql = """
INSERT INTO orders (trade, name, qty, price, active,
reason, market, xdate)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)"""
print(sql)


INSERT INTO orders (trade, name, qty, price, active,
reason, market, xdate)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)


In [26]:
for rcd in rcds:
    conlite.execute(sql, rcd)

In [27]:
sql = """
SELECT * FROM orders"""
df_orders = pd.read_sql(sql, conlite)
df_orders

,id,trade,name,qty,price,active,reason,market,xdate
0,1,B,CPNREIT,10000,18.80,0,HD,SET,2022-08-17
1,2,B,ASK,3000,33.50,0,52WL,SET,2022-03-03
2,3,B,BAM,6000,16.00,0,XXX,SET100,2022-04-29
3,4,B,BANPU,7500,13.00,0,CP1S,SET50,2022-09-14
4,5,B,BCP,3000,29.00,0,XXX,SET100,NaT
5,6,B,BLA,4000,33.00,0,1L,SET50,2022-08-23
6,7,B,JMART,3000,48.00,0,1L,SET50,2022-04-19
7,8,B,JMT,1200,73.00,0,1L,SET50,2022-04-19
8,9,B,KSL,30000,3.30,0,6L,SET,NaT
9,10,B,LANNA,3000,20.80,0,3L,SET,NaT


In [28]:
df_orders.loc[(df_orders.xdate == 'NaT'),['xdate']] = ['2022-02-02']
df_orders

,id,trade,name,qty,price,active,reason,market,xdate
0,1,B,CPNREIT,10000,18.80,0,HD,SET,2022-08-17
1,2,B,ASK,3000,33.50,0,52WL,SET,2022-03-03
2,3,B,BAM,6000,16.00,0,XXX,SET100,2022-04-29
3,4,B,BANPU,7500,13.00,0,CP1S,SET50,2022-09-14
4,5,B,BCP,3000,29.00,0,XXX,SET100,2022-02-02
5,6,B,BLA,4000,33.00,0,1L,SET50,2022-08-23
6,7,B,JMART,3000,48.00,0,1L,SET50,2022-04-19
7,8,B,JMT,1200,73.00,0,1L,SET50,2022-04-19
8,9,B,KSL,30000,3.30,0,6L,SET,2022-02-02
9,10,B,LANNA,3000,20.80,0,3L,SET,2022-02-02


In [29]:
pd.read_sql_query('SELECT * FROM orders', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,1,B,CPNREIT,10000,18.80,0,HD,SET,2022-08-17
1,2,B,ASK,3000,33.50,0,52WL,SET,2022-03-03
2,3,B,BAM,6000,16.00,0,XXX,SET100,2022-04-29
3,4,B,BANPU,7500,13.00,0,CP1S,SET50,2022-09-14
4,5,B,BCP,3000,29.00,0,XXX,SET100,NaT
5,6,B,BLA,4000,33.00,0,1L,SET50,2022-08-23
6,7,B,JMART,3000,48.00,0,1L,SET50,2022-04-19
7,8,B,JMT,1200,73.00,0,1L,SET50,2022-04-19
8,9,B,KSL,30000,3.30,0,6L,SET,NaT
9,10,B,LANNA,3000,20.80,0,3L,SET,NaT


In [30]:
sqlUpd = """
UPDATE orders
SET xdate = '2022-02-02'
WHERE xdate = 'NaT'"""
rp = conlite.execute(sqlUpd)
rp.rowcount

8

In [31]:
pd.read_sql_query('SELECT * FROM orders WHERE qty = 0', conlite)

,id,trade,name,qty,price,active,reason,market,xdate


In [32]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(box_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(one_file, header=True, index=False)

### Process to set target price

### 1) Set50 records

In [33]:
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET50" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,4,B,BANPU,7500,13.00,0,CP1S,SET50,2022-09-14
1,6,B,BLA,4000,33.00,0,1L,SET50,2022-08-23
2,7,B,JMART,3000,48.00,0,1L,SET50,2022-04-19
3,8,B,JMT,1200,73.00,0,1L,SET50,2022-04-19
4,13,B,PTT,4500,35.00,0,1L,SET50,2022-09-28
5,18,B,TOP,3000,54.00,0,1L,SET50,2022-02-02
6,28,S,IVL,2400,44.25,0,DOS,SET50,2022-08-30
7,20,S,PTTGC,3000,51.50,0,C51.25,SET50,2022-09-02


In [34]:
name = 'BLA'
limit = 20 # 1 month of data
sql = """
SELECT *
FROM price
WHERE name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe()

,price,maxp,minp,qty,opnp
count,20.00000,20.000000,20.000000,2.000000e+01,20.000000
mean,34.82500,35.412500,34.350000,1.172886e+07,34.900000
std,1.19786,1.316082,1.176525,7.466507e+06,1.133811
min,32.75000,33.500000,32.500000,3.658396e+06,33.000000
25%,33.68750,33.937500,33.187500,6.257906e+06,33.687500
50%,35.25000,35.875000,34.750000,8.796625e+06,35.250000
75%,35.56250,36.312500,35.250000,1.465224e+07,35.750000
max,37.00000,37.500000,36.000000,3.309337e+07,36.500000


### 2) Set100 records

In [35]:
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET100" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,3,B,BAM,6000,16.0,0,XXX,SET100,2022-04-29
1,5,B,BCP,3000,29.0,0,XXX,SET100,2022-02-02
2,12,B,PSL,6000,14.0,0,3L,SET100,2022-02-02
3,14,B,SINGER,3000,35.5,0,52WL,SET100,2022-02-02
4,15,B,SPALI,6000,18.5,0,3L,SET100,2022-08-23
5,17,B,STARK,25000,3.9,0,3L,SET100,2022-02-02
6,23,S,BCH,3000,20.6,0,C20.50,SET100,2022-08-31
7,24,S,DOHOME,2800,20.3,0,RD20pct,SET100,2022-02-02
8,22,S,STA,2500,26.0,0,CP1S,SET100,2022-08-23


In [36]:
name = 'KKP'
limit = 60 # 1 month of data
sql = """
SELECT *
FROM price
WHERE name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe()

,price,maxp,minp,qty,opnp
count,60.000000,60.000000,60.000000,6.000000e+01,60.000000
mean,66.983333,67.570833,66.370833,5.049026e+06,66.941667
std,3.782449,3.710410,3.752539,2.420507e+06,3.677222
min,60.500000,61.500000,59.500000,1.643975e+06,60.750000
25%,63.000000,63.500000,62.750000,3.419184e+06,63.187500
50%,67.875000,68.375000,67.250000,4.427620e+06,68.000000
75%,70.750000,71.250000,69.812500,5.734829e+06,70.125000
max,72.500000,72.500000,71.500000,1.671377e+07,72.250000


### 3) Set records

In [37]:
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,2,B,ASK,3000,33.50,0,52WL,SET,2022-03-03
1,1,B,CPNREIT,10000,18.80,0,HD,SET,2022-08-17
2,9,B,KSL,30000,3.30,0,6L,SET,2022-02-02
3,10,B,LANNA,3000,20.80,0,3L,SET,2022-02-02
4,11,B,OISHI,2400,42.00,0,6L,SET,2022-02-02
5,16,B,SSP,6000,8.50,0,52WL,SET,2022-02-02
6,26,S,ASP,10000,3.82,0,CP1S,SET,2022-08-24
7,29,S,GVREIT,10000,9.20,0,DOS,SET,2022-08-26
8,21,S,JASIF,10000,9.20,0,CP1S,SET,2022-08-19
9,19,S,MCS,25000,12.90,0,C12.8,SET,2022-09-03


In [38]:
name = 'AIT'
limit = 120 # 1 month of data
sql = """
SELECT *
FROM price
WHERE name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe()

,price,maxp,minp,qty,opnp
count,120.000000,120.000000,120.000000,1.200000e+02,120.000000
mean,6.507917,6.642083,6.419167,8.166416e+06,6.522917
std,0.550668,0.577884,0.538719,9.851153e+06,0.558769
min,5.550000,5.650000,5.500000,8.111610e+05,5.550000
25%,6.050000,6.150000,5.987500,2.111686e+06,6.100000
50%,6.525000,6.675000,6.350000,4.568450e+06,6.550000
75%,6.900000,7.050000,6.750000,8.666010e+06,6.912500
max,8.050000,8.250000,7.900000,4.831210e+07,8.050000
